In [1]:
import numpy as np
import socket
from PIL import Image
from keras.models import load_model
%matplotlib inline

width = 64
height = 64
channel = 3

Using TensorFlow backend.


## Load the models

In [2]:
lidar_model = load_model('car_model_lidar.h5')
camera_model = load_model('car_model_camera.h5')

## Class for environment & agent interactions

In [3]:
class EnvironmentInteraction:
    
    def __init__(self, Address, Port):
        
        # This byte array like delimeter uses to divide input byte socket stream to camera and lidar image
        self.byteDelimeter = b'\xff' * 4 + b'\x00' * 4 + b'\xff' * 4 + b'\x00' * 4 + b'\xff' * 4
        
        # The ip address of Unity client, usually set to null
        self.address = Address
        
        # The port to listen
        self.port = Port
        
        # The socket object
        self.sock = socket.socket()
        
        # Init socket object
        self.sock.bind((self.address, self.port))
            
        
    def listen(self):
        
        # Start to listen
        self.sock.listen(1)
        
        # Store connection when get request
        self.connection, self.address = self.sock.accept()
        
        
    def step(self, action):
        
        # Send the action to environment
        self.connection.send(action.encode('utf-8'))                        # Send action to environment
        
        # Receive the next images
        data = self.connection.recv(768432)                                 # Get camera and lidar images as byte array
        
        # Split to two byte arrays
        data = data.split(self.byteDelimeter)
        
        # Save the byte arrays as images
        camera = Image.frombytes('RGB', (64, 64), data[0])
        lidar = Image.frombytes('RGB', (64, 64), data[1])
        
        # Convert to numpy ndarray with 64x64x3 shape to flip images
        camera = np.array(camera.getdata(), dtype = np.float32).reshape(width, height, channel)
        lidar = np.array(lidar.getdata(), dtype = np.float32).reshape(width, height, channel)
        
        # Normalize the data
        camera = camera / 255
        lidar = lidar / 255
        
        # Flip the images horizontally
        camera = np.flip(camera, axis = 0)    
        lidar = np.flip(lidar, axis = 0)
        
        # Convert to numpy ndarray with 1x64x64x3 shape to predict by models
        camera = np.array(camera, dtype = np.float32).reshape(1, width, height, channel)
        lidar = np.array(lidar, dtype = np.float32).reshape(1, width, height, channel)
                
        return camera, lidar

# Object of EnvironmentInteraction class
environment = EnvironmentInteraction('', 7777) 

In [4]:
lidar_target_names = ['Human', 'Clear path', 'Obstacle']
lane_target_names = ['Left Lane', 'Right Lane']
counter = 0
obstacleDetected = False

# Function for formatting the command to environment
# The control of car in the environment makes by only one number, angle of turning
def command(lane_output, angle_output, lidar_output):
    
    global lidar_target_names
    global lane_target_names
    global counter
    global obstacleDetected
    
    action = '0'
    
    
    if counter == 15:
        counter = 0
        obstacleDetected = False
    
    # This condition is uses to give the car time to go over the next lane 
    if ((obstacleDetected) & (lidar_target_names[lidar_output] != 'Obstacle')):
        counter += 1
        return '0'
    
    # The main priority gives to lidar model, rather than camera model
    # Depending on what object is detected, system will make decision how to control the car
    if lidar_target_names[lidar_output] == 'Human':
        action = '-999'     # Stop moving
    elif lidar_target_names[lidar_output] == 'Obstacle':
        obstacleDetected = True
        if lane_target_names[lane_output] == 'Left Lane':
            action = '45'   # move to right lane
        else:
            action = '-45'  # move to left lane
    else:
        action = str(angle_output)
    
    return action

In [5]:
environment.listen()
print(environment.connection)

action = '0#_#_'

while(True):
        
    # Send the action and get next images to predict
    cameraImage, lidarImage = environment.step(action)
    
    # Make predictions
    camera_output = camera_model.predict(cameraImage)
    lidar_output = lidar_model.predict(lidarImage)
    
    # Calculate turning angle
    angle = camera_output[1][0][0] * 60
    
    # Form a new command
    action = command(np.argmax(camera_output[0]), angle, np.argmax(lidar_output)) + '#' + lane_target_names[np.argmax(camera_output[0])] + '#' + lidar_target_names[np.argmax(lidar_output)]
    
    print(action)

<socket.socket fd=1016, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 7777), raddr=('127.0.0.1', 50863)>
-0.843402519822#Left Lane#Clear path
-0.843402519822#Left Lane#Clear path
-0.843402519822#Left Lane#Clear path
-0.679503455758#Left Lane#Clear path
-0.550485178828#Left Lane#Clear path
-1.41116626561#Left Lane#Clear path
-0.974609479308#Left Lane#Clear path
-0.378588242456#Left Lane#Clear path
0.52327953279#Left Lane#Clear path
0.530027970672#Left Lane#Clear path
0.983944013715#Left Lane#Clear path
1.25541128218#Left Lane#Clear path
1.25986777246#Left Lane#Clear path
1.09451137483#Left Lane#Clear path
0.746555998921#Left Lane#Clear path
-0.119078261778#Left Lane#Clear path
-0.468753352761#Left Lane#Clear path
-0.106843358371#Left Lane#Clear path
-0.628237947822#Left Lane#Clear path
-1.46718330681#Left Lane#Clear path
-1.43625833094#Left Lane#Clear path
-1.64100833237#Left Lane#Clear path
-0.390343917534#Left Lane#Clear path
0.999102517962#Le

-3.30639995635#Right Lane#Clear path
-3.48804987967#Right Lane#Clear path
-0.597466304898#Right Lane#Clear path
2.9121197015#Right Lane#Clear path
3.32158349454#Right Lane#Clear path
0.746445581317#Right Lane#Clear path
-0.379082662985#Right Lane#Clear path
-0.0513936043717#Right Lane#Clear path
-3.78553748131#Right Lane#Clear path
-5.25443851948#Right Lane#Clear path
-6.64806157351#Right Lane#Clear path
-7.12843626738#Right Lane#Clear path
0.0214319466613#Right Lane#Clear path
1.28023467958#Right Lane#Clear path
7.36606925726#Right Lane#Clear path
7.81399279833#Right Lane#Clear path
6.39476627111#Right Lane#Clear path
2.07423783839#Right Lane#Clear path
1.54938451946#Right Lane#Clear path
-2.00872667134#Right Lane#Clear path
-3.18999119103#Right Lane#Clear path
-6.76916241646#Right Lane#Clear path
-8.19953262806#Right Lane#Clear path
-8.80782008171#Right Lane#Clear path
-10.5770015717#Right Lane#Clear path
-5.96365213394#Right Lane#Clear path
6.44927993417#Right Lane#Clear path
3.7177

-3.31057526171#Right Lane#Clear path
-5.30578717589#Right Lane#Clear path
-5.33198833466#Right Lane#Clear path
-1.69563107193#Right Lane#Clear path
2.19942457974#Right Lane#Clear path
3.66378240287#Right Lane#Clear path
5.19781261683#Right Lane#Clear path
3.36747072637#Right Lane#Clear path
-0.519260242581#Right Lane#Clear path
-1.10034517944#Right Lane#Clear path
-3.65387253463#Right Lane#Clear path
-6.19870707393#Right Lane#Clear path
-3.3292657882#Right Lane#Clear path
0.242897914723#Right Lane#Clear path
-2.05492459238#Right Lane#Clear path
-1.66581429541#Right Lane#Clear path
-0.956113412976#Right Lane#Clear path
-4.27984803915#Right Lane#Clear path
-2.26392902434#Right Lane#Clear path
-5.32080709934#Right Lane#Clear path
-9.50358241796#Right Lane#Clear path
-7.38912016153#Right Lane#Clear path
-9.69264060259#Right Lane#Clear path
-1.80261187255#Right Lane#Clear path
-4.05346333981#Right Lane#Clear path
-0.127482069656#Right Lane#Clear path
2.86629103124#Right Lane#Clear path
5.39

-11.2445989251#Right Lane#Clear path
2.48804055154#Right Lane#Clear path
-0.0340075255372#Right Lane#Clear path
3.6619245261#Right Lane#Clear path
2.17098020017#Right Lane#Clear path
3.93487930298#Right Lane#Clear path
3.94946873188#Right Lane#Clear path
4.0013153851#Right Lane#Clear path
2.56553225219#Right Lane#Clear path
2.83761061728#Right Lane#Clear path
-3.25940944254#Right Lane#Clear path
-8.14269036055#Right Lane#Clear path
-6.38539999723#Right Lane#Clear path
-3.52912522852#Right Lane#Clear path
0.400013225153#Right Lane#Clear path
4.78335231543#Right Lane#Clear path
3.53956080973#Right Lane#Clear path
1.76965422928#Right Lane#Clear path
1.9631832093#Right Lane#Clear path
-0.49215786159#Right Lane#Clear path
-7.64119416475#Right Lane#Clear path
-9.4220790267#Right Lane#Clear path
-9.35194194317#Right Lane#Clear path
-0.350473327562#Right Lane#Clear path
5.81417173147#Right Lane#Clear path
-1.40660203993#Right Lane#Clear path
-0.723100081086#Right Lane#Clear path
-0.10229030856

ValueError: not enough image data